In [120]:
import os
import numpy as np
import pandas as pd
import requests
import json
import time

# Stock scraping

### Functions

In [121]:
def get_sp500_list():
    """
    Get a list of S&P 500 company symbols by reading the Wikipedia page.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_table = tables[0]
    sp500_symbols = sp500_table["Symbol"].tolist()
    return sp500_symbols

### Constants

In [122]:
API_KEY_ALPHAVANTAGE = '76I7ZXLX7S7BSRAX'
API_KEY_FAM = 'NdLNkc2mzBnSVEvOgyOqB3CBGN4YBm4v'

BASE_URL_FAM = 'https://financialmodelingprep.com/api/v3'

REQUEST_LIMIT = 250
req_count = 0
YEARS = 15

FIN_DATA_DIR = '../financial_data/'
DATA_DIR = '../data/'

# List of required metrics
required_metrics = [
    "GrossProfit",
    "Revenues",
    "NetIncomeLoss",
    "StockholdersEquity",
    "Liabilities",
    "AssetsNoncurrent",
    "NetCashProvidedByUsedInOperatingActivities"
]

## Data collection

In [123]:
# This is the list of available tickers using the free FMP subsribtion
FAM_free_access = [
    'AAPL', 'TSLA', 'AMZN', 'MSFT', 'NVDA', 'GOOGL', 'META', 'NFLX', 'JPM', 'V', 'BAC', 'AMD', 'PYPL', 'DIS', 'T', 'PFE', 'COST', 'INTC', 'KO', 'TGT', 'NKE', 'SPY', 'BA', 'BABA', 'XOM', 'WMT', 'GE', 'CSCO', 'VZ', 'JNJ', 'CVX', 'PLTR', 'SQ', 'SHOP', 'SBUX', 'SOFI', 'HOOD', 'RBLX', 'SNAP', 'UBER', 'FDX', 'ABBV', 'ETSY', 'MRNA', 'LMT', 'GM', 'F', 'RIVN', 'LCID', 'CCL', 'DAL', 'UAL', 'AAL', 'TSM', 'SONY', 'ET', 'NOK', 'MRO', 'COIN', 'SIRI', 'RIOT', 'CPRX', 'VWO', 'SPYG', 'ROKU', 'VIAC', 'ATVI', 'BIDU', 'DOCU', 'ZM', 'PINS', 'TLRY', 'WBA', 'MGM', 'NIO', 'C', 'GS', 'WFC', 'ADBE', 'PEP', 'UNH', 'CARR', 'FUBO', 'HCA', 'TWTR', 'BILI', 'RKT'
]

In [124]:
# Create .csv files to keep track of the company data that I have and still need to get
if not os.path.exists(f"{DATA_DIR}sp500_tickers.csv"):
    sp500_list = get_sp500_list()
    df_sp500 = pd.DataFrame(sp500_list)
    df_sp500.to_csv(f"{DATA_DIR}sp500_tickers.csv", index=False)
else:
    print("sp500_tickers.csv already exists, therefore import it.")
    df_sp500 = pd.read_csv(f"{DATA_DIR}sp500_tickers.csv")

if not os.path.exists(f"{DATA_DIR}data_progress_sp500.csv"):
    df_progress = pd.DataFrame(columns=['ticker', 'income_done', 'balance_done', 'cashflow_done'])
    df_progress['ticker'] = sp500_list
    df_progress['income_done'] = False
    df_progress['balance_done'] = False
    df_progress['cashflow_done'] = False
    df_progress.set_index('ticker', inplace=True)
    df_progress.to_csv(f"{DATA_DIR}data_progress_sp500.csv")
    df_data_progress = df_progress
else: 
    print("data_progress_sp500.csv already exists, therefore import it.")
    df_data_progress_sp500 = pd.read_csv(f"{DATA_DIR}data_progress_sp500.csv", index_col='ticker')

if not os.path.exists(f"{DATA_DIR}data_progress_free.csv"):
    df_progress = pd.DataFrame(columns=['ticker', 'income_done', 'balance_done', 'cashflow_done'])
    df_progress['ticker'] = FAM_free_access
    df_progress['income_done'] = False
    df_progress['balance_done'] = False
    df_progress['cashflow_done'] = False
    df_progress.set_index('ticker', inplace=True)
    df_progress.to_csv(f"{DATA_DIR}data_progress_free.csv")
    df_data_progress_free = df_progress
else: 
    print("data_progress_free.csv already exists, therefore import it.")
    df_data_progress_free = pd.read_csv(f"{DATA_DIR}data_progress_free.csv", index_col='ticker')

sp500_tickers.csv already exists, therefore import it.
data_progress_sp500.csv already exists, therefore import it.
data_progress_free.csv already exists, therefore import it.


In [125]:
print(df_data_progress_free)
print(df_data_progress_free.info())

       income_done balance_done cashflow_done
ticker                                       
AAPL          True         True          True
TSLA          True         True          True
AMZN          True         True          True
MSFT          True         True          True
NVDA          True         True          True
...            ...          ...           ...
FUBO          True         True          True
HCA           True         True          True
TWTR          True         True          True
BILI          True         True          True
RKT           True         True          True

[87 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
Index: 87 entries, AAPL to RKT
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   income_done    87 non-null     object
 1   balance_done   87 non-null     object
 2   cashflow_done  87 non-null     object
dtypes: object(3)
memory usage: 2.7+ KB
None


In [126]:

list_tickers = df_data_progress_sp500.index.tolist()

# check if the available tickers is in the S&P500 list
for ticker in FAM_free_access:
    if ticker not in list_tickers:
        print(f"{ticker} is not in the s&p500 list")
        pass


SPY is not in the s&p500 list
BABA is not in the s&p500 list
SQ is not in the s&p500 list
SHOP is not in the s&p500 list
SOFI is not in the s&p500 list
HOOD is not in the s&p500 list
RBLX is not in the s&p500 list
SNAP is not in the s&p500 list
ETSY is not in the s&p500 list
RIVN is not in the s&p500 list
LCID is not in the s&p500 list
AAL is not in the s&p500 list
TSM is not in the s&p500 list
SONY is not in the s&p500 list
ET is not in the s&p500 list
NOK is not in the s&p500 list
MRO is not in the s&p500 list
SIRI is not in the s&p500 list
RIOT is not in the s&p500 list
CPRX is not in the s&p500 list
VWO is not in the s&p500 list
SPYG is not in the s&p500 list
ROKU is not in the s&p500 list
VIAC is not in the s&p500 list
ATVI is not in the s&p500 list
BIDU is not in the s&p500 list
DOCU is not in the s&p500 list
ZM is not in the s&p500 list
PINS is not in the s&p500 list
TLRY is not in the s&p500 list
NIO is not in the s&p500 list
FUBO is not in the s&p500 list
TWTR is not in the s&

In [127]:
drop_income_cols = ['reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'calendarYear', 'link', 'finalLink', 'eps', 'epsdiluted', 'weightedAverageShsOut', 'weightedAverageShsOutDil', 'grossProfitRatio', 'ebitdaratio', 'operatingIncomeRatio', 'incomeBeforeTaxRatio']

drop_balance_columns = ['reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'calendarYear', 'finalLink']

drop_cashflow_columns = ['reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'calendarYear', 'finalLink']


## Get the statements

Retreive the Income, Balance and Cashflow statements for all the companies
For the time being only get the statements of the companies that I have free access to, but will late change this to all of the companies on the S&P500. 

Also try to combine the statements when I get them inputted

In [128]:
df_data_progress_free.dtypes

income_done      object
balance_done     object
cashflow_done    object
dtype: object

In [129]:
list_tickers = df_data_progress_free.index.tolist()

# Loop through the tickers and get their income, balance and cashflow statements
for ticker in list_tickers:
    
    # Ensure the correct amount of API calls
    if not ((REQUEST_LIMIT - req_count) > 3):
        print(f"Request limit reached, curr count: {req_count}")
        break

    row = df_data_progress_free.loc[ticker]

    ### INCOME STATEMENT ###
    if (row['income_done'] == False):
        print(f"Getting income statement for {ticker}...")
        url = f"{BASE_URL_FAM}/income-statement/{ticker}?period=annual&limit={YEARS}&apikey={API_KEY_FAM}"
        
        try:
            response = requests.get(url)
            req_count += 1
        except Exception as e:
            print(f"Request failed for {ticker}: {e}")
            break

        print(f"Status Code: {response.status_code}")
        if response.status_code != 200:
            print(f"Error fetching {ticker}: {response.text}")
            break

        if response.ok:
            data = response.json()
            if data:
                df = pd.DataFrame(data)
                df = df.drop(columns=drop_income_cols)
                df.to_csv(f"{FIN_DATA_DIR}{ticker}_income.csv", index=False)
                df_data_progress_free.loc[ticker, 'income_done'] = True
                print(f"Income downloaded for {ticker}")
            else:
                print(f"No data returned for {ticker}")
                df_data_progress_free.loc[ticker, 'income_done'] = 'no_data'
        else:
            print(f"Failed to get income for {ticker}")
        time.sleep(1)

    ### BALANCE STATEMENT ###
    if (row['balance_done'] == False):
        print(f"Getting balance statement for {ticker}...")
        url = f"{BASE_URL_FAM}/balance-sheet-statement/{ticker}?limit={YEARS}&apikey={API_KEY_FAM}"
        
        # Ensure the request through the api works
        try:
            response = requests.get(url)
            req_count += 1
        except Exception as e:
            print(f"Request failed for {ticker}: {e}")
            break

        print(f"Status Code: {response.status_code}")
        if response.status_code != 200:
            print(f"Error fetching balance sheet for {ticker}: {response.text}")
        elif response.ok:
            data = response.json()
            if data:
                df = pd.DataFrame(data).drop(columns=drop_balance_columns)
                df.to_csv(f"{FIN_DATA_DIR}{ticker}_balance.csv", index=False)
                df_data_progress_free.loc[ticker, 'balance_done'] = True
                print(f"Balance sheet downloaded for {ticker}")
            else:
                print(f"No data returned for {ticker}")
                df_data_progress_free.loc[ticker, 'balance_done'] = 'no_data'
        time.sleep(1)

    ### CASHFLOW STATEMENT ###
    if (row['cashflow_done'] == False):
        print(f"Getting Cash flow statement for {ticker}...")
        url = f"{BASE_URL_FAM}/cash-flow-statement/{ticker}?limit={YEARS}&apikey={API_KEY_FAM}"

        # Ensure correctness of API call
        try:
            response = requests.get(url)
            req_count += 1
        except Exception as e:
            print(f"Request failed for {ticker}: {e}")
            break

        print(f"Status Code: {response.status_code}")
        if response.status_code != 200:
            print(f"Error fetching cash flow for {ticker}: {response.text}")
        elif response.ok:
            data = response.json()
            if data:
                df = pd.DataFrame(data).drop(columns=drop_cashflow_columns)
                df.to_csv(f"{FIN_DATA_DIR}{ticker}_cashflow.csv", index=False)
                df_data_progress_free.loc[ticker, 'cashflow_done'] = True
                print(f"Cash flow downloaded for {ticker}")
            else:
                print(f"No data returned for {ticker}")
                df_data_progress_free.loc[ticker, 'cashflow_done'] = 'no_data'
        time.sleep(1)
        print()

        
df_data_progress_free.to_csv(f"{DATA_DIR}data_progress_free.csv")

## Combine all of the statements

In [130]:
# Combine all of the data that was collected
combined_data = []
list_tickers = df_data_progress_free.index.tolist()
count = 0

if not os.path.exists("combined_data.csv"):
    df_progress = pd.read_csv(f"{DATA_DIR}data_progress_free.csv", index_col='ticker')
    for ticker in list_tickers:

        # limit for testing
        if (count > 1): break
            
        try:
            income_df = pd.read_csv(f"{FIN_DATA_DIR}{ticker}_income.csv")
            balance_df = pd.read_csv(f"{FIN_DATA_DIR}{ticker}_balance.csv")
            cashflow_df = pd.read_csv(f"{FIN_DATA_DIR}{ticker}_cashflow.csv")

            # Merge all three on 'date'
            df_merged = income_df.merge(balance_df, on='date', suffixes=('', '_bal'))
            df_merged = df_merged.merge(cashflow_df, on='date', suffixes=('', '_cf'))

            df_merged['ticker'] = ticker
            combined_data.append(df_merged)

        except Exception as e:
            print(f"Skipping {ticker} due to error: {e}")
        count +=1

    df_full = pd.concat(combined_data, ignore_index=True)
    df_full.to_csv("combined_data.csv", index=False)
else:
    print("combined_data.csv already exists")

